# stackoverflow posts load
### 参考
1. [python-es-bulk](https://github.com/elastic/elasticsearch-py/issues/508)
2. [python-es-bulk1](http://elasticsearch-py.readthedocs.io/en/master/helpers.html)
3. [parse-tags](https://tutorialedge.net/python/removing-html-from-string/)
4. [iteratively-parse-large-xml](http://effbot.org/zone/element-iterparse.htm)
5. [stackoverflow-data](https://archive.org/details/stackexchange)
6. [stackoverflow-data-analysis-online](http://data.stackexchange.com/stackoverflow/query/new)
7. [stackoverflow-data-google-bigquery](https://cloud.google.com/bigquery/public-data/stackoverflow)

In [1]:
import sys
sys.path.append("./")
import ElasticSearchClass
import importlib
importlib.reload(ElasticSearchClass)

import time
import xml.etree.ElementTree as ET
import math
import pandas as pd
from bs4 import BeautifulSoup

FIELDS = ['id', 'acceptedanswerid', 'creationdate', 'score', 'viewcount', 'body', 'owneruserid', 
          'lasteditoruserid', 'lasteditordisplayname', 'lasteditdate', 'lastactivitydate', 'title',
          'tags', 'answercount', 'commentcount', 'favoritecount', 'communityowneddate']
INDEX_MAPPING= '''
{
    "settings": {
            "index": {
                "refresh_interval": "5s",
                "number_of_shards": "20",
                "translog": {
                    "durability": "async"
                },
                "number_of_replicas": "1"
            }
        },
    "mappings": {
            "posts": {
                "properties": {
                    "acceptedanswerid": {
                        "type": "integer"
                    },
                    "answercount": {
                        "type": "integer"
                    },
                    "body": {
                        "type": "text",
                        "norms": false
                    },
                    "commentcount": {
                        "type": "integer"
                    },
                    "communityowneddate": {
                        "type": "date"
                    },
                    "creationdate": {
                        "type": "date"
                    },
                    "favoritecount": {
                        "type": "integer"
                    },
                    "lastactivitydate": {
                        "type": "date"
                    },
                    "lasteditdate": {
                        "type": "date"
                    },
                    "lasteditordisplayname": {
                        "type": "keyword"
                    },
                    "lasteditoruserid": {
                        "type": "integer"
                    },
                    "ownerdisplayname": {
                        "type": "keyword"
                    },
                    "owneruserid": {
                        "type": "integer"
                    },
                    "parentid": {
                        "type": "integer"
                    },
                    "posttypeid": {
                        "type": "integer"
                    },
                    "score": {
                        "type": "integer"
                    },
                    "tags": {
                        "type": "keyword"
                    },
                    "title": {
                        "type": "text",
                        "term_vector": "yes",
                        "fields": {
                            "keyword": {
                                "type": "keyword",
                                "ignore_above": 256
                            }
                        }
                    },
                    "viewcount": {
                        "type": "integer"
                    }
                }
            }
        }
}
'''
DOCID="docId" #stackoverflow的数据字段有一个为id, python中有名为id的内置函数，因此要修改这个字段名

In [3]:
def parseRow(elem):
    """
    Parameters in:
        elem - XML element
    Return:
        dictionary - key, value
    """
    values=dict()
    for key in elem.attrib.keys():
        if key.lower() == "tags":
            tags = elem.attrib.get(key).replace('<', '').split('>')[:-1]
            values[key.lower()] = tags
        elif key.lower() == "body":
            body = BeautifulSoup(elem.attrib.get(key), "html.parser").get_text()
            values[key.lower()] = body
        elif key.lower() == "id": # change field name to docID
            values[DOCID] = elem.attrib.get(key)
        else:
            values[key.lower()] = elem.attrib.get(key)
    return values

In [4]:
def formatOneTextRow(fields, values = None, fieldDLM="|@|"):
    """
    Parameters in:
        fields - a list of fields name
        values - a dictionary of values with fields name, Default None
        fieldDLM - delimeter of values, default "|@|"
    Return:
        if values is None, return fields with delimeters, used as header
        if is not None, values with delimeters in the same order as fields
    """
    #format a header
    if values is None:
        return fieldDLM.join(fields)
    
    #format a data row
    valueList = []
    for field in fields:
        if field in values:
            if field == "tags":
                valueList.append(",".join(values[field]))
            else:
                valueList.append(values[field])
        else: 
            valueList.append("")
    return fieldDLM.join(valueList)

In [5]:
def formatOneESIndex(values, indexName="stackoverflow", docType="posts"):
    if DOCID in values:
        docId = values[DOCID]
        del values[DOCID]
        indexRow = {
            "_index": indexName,
             "_type": docType,
             "_id":docId,
             "_source": values
        }
    else:
        indexRow =  {
            "_index": indexName,
            "_type": docType,
            "_source": values
         }
    return indexRow

In [6]:
def parseXML(XMLFile, resType="File", indexName="stackoverflow", docType="posts"):
    """
    Parameters in:
        XMLFile - XML File path for parse
        resType - File, parse XMLFile to another file with delimiter seperated data row, default value
                  ES, parse XMLFile to ES index format
    """
    #get an iterable
    context = ET.iterparse(XMLFile, events=("start", "end"))
    #turn it into an iterator
    context = iter(context)
    #get the root element
    event, root = next(context)
    count = 0  
    for event, elem in context:
        if count > 1000:
            break
        if event == "end" and elem.tag == "row":
            count += 1
            values = parseRow(elem)
            if resType == "ES":
                yield formatOneESIndex(values, indexName, docType)
            else:
                yield formatOneTextRow(FIELDS, values)
            elem.clear()
        root.clear()
    print("Parse XML [%s] Done, total [%d] records!" % (XMLFile, count))

In [8]:
def parseXMLToCSV(XMLFile, outFile):
    try:
        with open(outFile, 'w', newline='', encoding='utf-8') as out:
            out.write(formatOneTextRow(FIELDS))
            out.write("\n")   
            generated = parseXML(XMLFile)
            count = 0
            print(outFile)
            for data in generated:
                out.write(data)
                out.write("\n")
                count += 1
        print("Parse XML [%s] and save to file [%s] Done, total [%d] records!" % (XMLFile, outFile, count))
    except FileNotFoundError:
        print("ERROR:Can not read XMLFile [%s] or CSVFile [%s], will do nothing!" % (XMLFile, outFile))
                
parseXMLToCSV("F:\\stackoverflow.com-Posts\\Posts.xml", ".\Posts.csv")  

.\Posts.csv
Parse XML [F:\stackoverflow.com-Posts\Posts.xml] Done, total [1001] records!
Parse XML [F:\stackoverflow.com-Posts\Posts.xml] and save to file [.\Posts.csv] Done, total [1001] records!


In [9]:
def parseXMLToES(XMLFile, esUtil, indexName="stackoverflow", docType="posts"):
    if esUtil is None or not esUtil.isValid():
        print("esUtil in invalid, will do nothing!")
        return
    
    try:
        esUtil.createIndex(indexName, INDEX_MAPPING)
    except Exception as e:
        print("ERROR: Create ES Index failed!")
        print(str(e))
        return
    
    indexedCount = 0
    try:
        indexedCount = esUtil.bulkIndexDocument(parseXML(XMLFile, "ES", indexName, docType))
    except Exception as e:
        print(e)
    print("Index to ES Done, [%d] was indexed!" % (indexedCount))

In [11]:
if __name__=='__main__':
    #如果虚拟机内可以访问端口，宿主机访问不了，把虚拟机的firewall关掉试试，sudo service iptables stop
    esUtil = ElasticSearchClass.ElasticSearchClass("192.168.18.187", 9201)
    parseXMLToES("F:\\stackoverflow.com-Posts\\Posts.xml", esUtil)

Parse XML [F:\stackoverflow.com-Posts\Posts.xml] Done, total [1001] records!
Index to ES Done, [1001] was indexed!
